# **Air Quality**

---
---

# Data Pre-processing

In [36]:
# Import of libraries
import pandas as pd
#pip install pandas openpyxl (para leer excels)
import numpy as np
import folium
import matplotlib.pyplot as plt
import seaborn as sns

#### **Upload and view the polluting data for Spain in 2020**
- Concatenate the files to be able to work better with them.
- Process the data before the EDA.

In [37]:
C6H6_HH_22 = pd.read_csv('../data/raw/time data 2022/C6H6_HH_2022.csv', sep = ';')
CO_HH_22 = pd.read_csv('../data/raw/time data 2022/CO_HH_2022.csv', sep= ';')
NO2_HH_22 = pd.read_csv('../data/raw/time data 2022/NO2_HH_2022.csv', sep = ';')
NOx_HH_22 = pd.read_csv('../data/raw/time data 2022/NOx_HH_2022.csv', sep = ';')
O3_HH_22 = pd.read_csv('../data/raw/time data 2022/O3_HH_2022.csv', sep = ';')
PM10_HH_22 = pd.read_csv('../data/raw/time data 2022/PM10_HH_2022.csv', sep = ';')
PM25_HH_22 = pd.read_csv('../data/raw/time data 2022/PM25_HH_2022.csv', sep = ';')
SO2_HH_22 = pd.read_csv('../data/raw/time data 2022/SO2_HH_2022.csv', sep = ';')

In [38]:
pollutants = pd.concat([C6H6_HH_22, CO_HH_22, NO2_HH_22, NOx_HH_22, O3_HH_22, PM10_HH_22, PM25_HH_22, SO2_HH_22])
print(pollutants.shape)

(819614, 32)


In [39]:
pollutants.head()

PROVINCIA  MUNICIPIO  ESTACION  MAGNITUD  PUNTO_MUESTREO  ANNO  MES  DIA  \
0          1         59         8        30  01059008_30_59  2022    1    1   
1          1         59         8        30  01059008_30_59  2022    1    3   
2          1         59         8        30  01059008_30_59  2022    1    4   
3          1         59         8        30  01059008_30_59  2022    1    5   
4          1         59         8        30  01059008_30_59  2022    1    6   

    H01   H02  ...   H15   H16   H17   H18   H19   H20   H21   H22   H23   H24  
0  1.58  0.73  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
1   NaN   NaN  ...  0.23  0.22  0.23  0.58  0.24  0.54  0.69  0.32  0.17  0.17  
2  0.16  0.16  ...  0.17  0.18  0.19  0.25  0.25  0.28  0.25  0.20  0.20  0.22  
3  0.23  0.26  ...  0.48  0.33  0.45  0.58  0.57  0.51  0.51  0.51  0.48  0.41  
4  0.36  0.31  ...  0.30  0.26  0.29  0.36  0.43  0.32  0.32  0.30  0.30  0.28  

[5 rows x 32 columns]

In [40]:
pollutants.to_csv('../data/interim/pollutants.csv', index= False)
pollutants = pd.read_csv('../data/interim/pollutants.csv')

In [41]:
total_nan = pollutants.isna().sum().sum()
total_nan

367595

In [42]:
row_nan = pollutants.isna().any(axis=1).sum()
row_nan

100332

In [43]:
# pollutants = pollutants.dropna()
# pollutants

In [44]:
measured_excel = pd.read_excel('../data/raw/Metainformacion2022.xlsx', sheet_name= 'Magnitudes')
measured_excel.head(14)

MAGNITUD                           NOMBRE ABREVIATURA     UNIDAD  \
0          1                DIÓXIDO DE AZUFRE         SO2      µg/m3   
1          6              MONÓXIDO DE CARBONO          CO      mg/m3   
2          7            MONÓXIDO DE NITRÓGENO          NO      µg/m3   
3          8             DIÓXIDO DE NITRÓGENO         NO2      µg/m3   
4          9  PARTÍCULAS EN SUSPENSIÓN <2,5µM       PM2.5      µg/m3   
5         10   PARTÍCULAS EN SUSPENSIÓN <10µM        PM10      µg/m3   
6         12      ÓXIDOS DE NITRÓGENO TOTALES         NOX  µg/m3 NO2   
7         14                            OZONO          O3      µg/m3   
8         17                  ARSENICO (PM10)          As      ng/m3   
9         19                     PLOMO (PM10)          Pb      µg/m3   
10        27            BENZO(A)PIRENO (PM10)         BAP      ng/m3   
11        28                    CADMIO (PM10)          Cd      ng/m3   
12        30                          BENCENO        C6H6      µg/m3   
13        62                    NÍQUEL (PM10)          NI      ng/m3   

                                GRUPO  
0   Contaminantes con valor legislado  
1   Contaminantes con valor legislado  
2                                 NaN  
3   Contaminantes con valor legislado  
4   Contaminantes con valor legislado  
5   Contaminantes con valor legislado  
6   Contaminantes con valor legislado  
7   Contaminantes con valor legislado  
8   Contaminantes con valor legislado  
9   Contaminantes con valor legislado  
10  Contaminantes con valor legislado  
11  Contaminantes con valor legislado  
12  Contaminantes con valor legislado  
13  Contaminantes con valor legislado

In [45]:
# Leer una hoja específica del archivo Excel
stations_excel= pd.read_excel('../data/raw/Metainformacion2022.xlsx', sheet_name='Estaciones')
stations_excel.head(3)

COD_LOCAL  PROVINCIA  MUNICIPIO  ESTACION COD_ESTACION_DEM  \
0    1022001          1         22         1          ES1672A   
1    1036004          1         36         4          ES1349A   
2    1051001          1         51         1          ES1544A   

             N_RED    NOMBRE  FECHA_INI FECHA_FIN  LATITUD_G  ...  ALTITUD  \
0  CCAA País Vasco  EL CIEGO 2004-05-10       NaT   42.51833  ...      480   
1  CCAA País Vasco    LLODIO 1994-01-01       NaT   43.14407  ...      122   
2  CCAA País Vasco   AGURAIN 1998-01-01       NaT   42.84900  ...      594   

       N_CCAA N_PROVINCIA            N_MUNICIPIO TIPO_ESTACION  TIPO_AREA  \
0  PAÍS VASCO       ÁLAVA                ELCIEGO       TRAFICO  SUBURBANA   
1  PAÍS VASCO       ÁLAVA                 LLODIO       TRAFICO  SUBURBANA   
2  PAÍS VASCO       ÁLAVA  SALVATIERRA O AGURAIN         FONDO  SUBURBANA   

  TIPO_SUBAREA_RURAL EST                    ZONA                    DIRECCION  
0                NaN  ST             RESIDENCIAL  C/GABRIEL CELAYA 8, ELCIEGO  
1                NaN  ST   RESIDENCIAL/COMERCIAL               C/ LAMUZA, S/N  
2                NaN  SF  RESIDENCIAL/INDUSTRIAL      CUARTEL DE LA ERTZANTZA  

[3 rows x 21 columns]

In [46]:
columnas_a_modificar = ['LATITUD_G', 'LONGITUD_G']

for columna in columnas_a_modificar:
    # Asegurarse de que todos los valores no-NaN sean strings
    # y aplicar la operación de reemplazo
    stations_excel[columna] = stations_excel[columna].apply(lambda x: str(x).replace(',', '.') if pd.notnull(x) else x)

# Después de asegurarse de que todos los valores son strings y de reemplazar las comas
# Convertimos toda la columna a float
for columna in columnas_a_modificar:
    stations_excel[columna] = stations_excel[columna].astype(float)

stations_excel.head()


COD_LOCAL  PROVINCIA  MUNICIPIO  ESTACION COD_ESTACION_DEM  \
0    1022001          1         22         1          ES1672A   
1    1036004          1         36         4          ES1349A   
2    1051001          1         51         1          ES1544A   
3    1055001          1         55         1          ES1489A   
4    1059008          1         59         8          ES1502A   

             N_RED           NOMBRE  FECHA_INI FECHA_FIN  LATITUD_G  ...  \
0  CCAA País Vasco         EL CIEGO 2004-05-10       NaT   42.51833  ...   
1  CCAA País Vasco           LLODIO 1994-01-01       NaT   43.14407  ...   
2  CCAA País Vasco          AGURAIN 1998-01-01       NaT   42.84900  ...   
3  CCAA País Vasco        VALDEREJO 1998-01-01       NaT   42.87520  ...   
4  CCAA País Vasco  AVENIDA GASTEIZ 1998-01-01       NaT   42.85480  ...   

   ALTITUD      N_CCAA N_PROVINCIA            N_MUNICIPIO TIPO_ESTACION  \
0      480  PAÍS VASCO       ÁLAVA                ELCIEGO       TRAFICO   
1      122  PAÍS VASCO       ÁLAVA                 LLODIO       TRAFICO   
2      594  PAÍS VASCO       ÁLAVA  SALVATIERRA O AGURAIN         FONDO   
3      911  PAÍS VASCO       ÁLAVA             VALDEGOVÍA         FONDO   
4      517  PAÍS VASCO       ÁLAVA        VITORIA-GASTEIZ       TRAFICO   

   TIPO_AREA TIPO_SUBAREA_RURAL    EST                    ZONA  \
0  SUBURBANA                NaN     ST             RESIDENCIAL   
1  SUBURBANA                NaN     ST   RESIDENCIAL/COMERCIAL   
2  SUBURBANA                NaN     SF  RESIDENCIAL/INDUSTRIAL   
3      RURAL             REMOTA  RFREM              NATURALEZA   
4     URBANA                NaN     UT   RESIDENCIAL/COMERCIAL   

                     DIRECCION  
0  C/GABRIEL CELAYA 8, ELCIEGO  
1               C/ LAMUZA, S/N  
2      CUARTEL DE LA ERTZANTZA  
3  PARQUE NATURAL DE VALDEREJO  
4            AVDA. GASTEIZ, 93  

[5 rows x 21 columns]

In [47]:
# Crear un mapa base
m = folium.Map(location=[stations_excel['LATITUD_G'].mean(), stations_excel['LONGITUD_G'].mean()], zoom_start=6)

# Añadir puntos al mapa
for idx, row in stations_excel.iterrows():
    folium.Marker(location=[row['LATITUD_G'], row['LONGITUD_G']],
                  popup=row['N_MUNICIPIO'] if 'NombrePunto' in stations_excel.columns else "Sin Nombre").add_to(m)

# Guardar el mapa en un archivo HTML
m.save('mapa.html')
#Aqui no se ve nada, creo que hay qeu hacerlo en un navegador normal
# Si descargamos en nuestro sistema y lo ejecutamos, abre directamente una web con el mapa creado

In [48]:
pollutants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 819614 entries, 0 to 819613
Data columns (total 32 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PROVINCIA       819614 non-null  int64  
 1   MUNICIPIO       819614 non-null  int64  
 2   ESTACION        819614 non-null  int64  
 3   MAGNITUD        819614 non-null  int64  
 4   PUNTO_MUESTREO  819614 non-null  object 
 5   ANNO            819614 non-null  int64  
 6   MES             819614 non-null  int64  
 7   DIA             819614 non-null  int64  
 8   H01             809146 non-null  float64
 9   H02             809010 non-null  float64
 10  H03             807769 non-null  float64
 11  H04             808729 non-null  float64
 12  H05             809302 non-null  float64
 13  H06             807322 non-null  float64
 14  H07             804602 non-null  float64
 15  H08             798578 non-null  float64
 16  H09             791779 non-null  float64
 17  H10       

In [49]:
pollutants.head()

PROVINCIA  MUNICIPIO  ESTACION  MAGNITUD  PUNTO_MUESTREO  ANNO  MES  DIA  \
0          1         59         8        30  01059008_30_59  2022    1    1   
1          1         59         8        30  01059008_30_59  2022    1    3   
2          1         59         8        30  01059008_30_59  2022    1    4   
3          1         59         8        30  01059008_30_59  2022    1    5   
4          1         59         8        30  01059008_30_59  2022    1    6   

    H01   H02  ...   H15   H16   H17   H18   H19   H20   H21   H22   H23   H24  
0  1.58  0.73  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
1   NaN   NaN  ...  0.23  0.22  0.23  0.58  0.24  0.54  0.69  0.32  0.17  0.17  
2  0.16  0.16  ...  0.17  0.18  0.19  0.25  0.25  0.28  0.25  0.20  0.20  0.22  
3  0.23  0.26  ...  0.48  0.33  0.45  0.58  0.57  0.51  0.51  0.51  0.48  0.41  
4  0.36  0.31  ...  0.30  0.26  0.29  0.36  0.43  0.32  0.32  0.30  0.30  0.28  

[5 rows x 32 columns]

In [50]:
# Crear una columna 'fecha' con el año, mes y día
pollutants['FECHA'] = pd.to_datetime(pollutants[['ANNO', 'MES', 'DIA']].astype(str).agg('-'.join, axis=1), errors='coerce', format='%Y-%m-%d')
pollutants.drop(["ANNO", "MES", "DIA"], axis=1, inplace=True)

In [51]:
# Crear diccionario de las magnitudes
measured_dict = {}

for index, row in measured_excel.iterrows():
    measured_dict[row['MAGNITUD']] = row['ABREVIATURA']

measured_dict

{1: 'SO2',
 6: 'CO',
 7: 'NO',
 8: 'NO2',
 9: 'PM2.5',
 10: 'PM10',
 12: 'NOX',
 14: 'O3',
 17: 'As',
 19: 'Pb',
 27: 'BAP',
 28: 'Cd',
 30: 'C6H6',
 62: 'NI'}

In [52]:
pollutants.drop(["PUNTO_MUESTREO"], axis=1, inplace=True)

In [53]:
# Suponiendo que las dos claves son 'CLAVE1' y 'CLAVE2' y el valor es 'VALOR'
diccionario_municipios = {(row['PROVINCIA'], row['MUNICIPIO']): row['N_MUNICIPIO'] for index, row in stations_excel.iterrows()}
diccionario_municipios


{(1, 22): 'ELCIEGO',
 (1, 36): 'LLODIO',
 (1, 51): 'SALVATIERRA O AGURAIN',
 (1, 55): 'VALDEGOVÍA',
 (1, 59): 'VITORIA-GASTEIZ',
 (2, 3): 'ALBACETE',
 (3, 2): 'AGOST',
 (3, 9): 'ALCOY/ALCOI',
 (3, 14): 'ALICANTE/ALACANT',
 (3, 31): 'BENIDORM',
 (3, 65): 'ELCHE/ELX',
 (3, 66): 'ELDA',
 (3, 99): 'ORIHUELA',
 (3, 105): 'PINOSO',
 (3, 133): 'TORREVIEJA',
 (4, 13): 'ALMERÍA',
 (4, 22): 'BÉDAR',
 (4, 24): 'BENAHADUX',
 (4, 32): 'CARBONERAS',
 (4, 35): 'CUEVAS DEL ALMANZORA',
 (4, 66): 'NÍJAR',
 (4, 902): 'EJIDO (EL)',
 (5, 19): 'ÁVILA',
 (6, 15): 'BADAJOZ',
 (6, 16): 'BARCARROTA',
 (6, 22): 'BURGUILLOS DEL CERRO',
 (6, 70): 'JEREZ DE LOS CABALLEROS',
 (6, 81): 'MEDINA DE LAS TORRES',
 (6, 83): 'MÉRIDA',
 (6, 158): 'ZAFRA',
 (7, 3): 'ALCÚDIA',
 (7, 10): 'BUNYOLA',
 (7, 15): 'CIUTADELLA DE MENORCA',
 (7, 19): 'ESCORCA',
 (7, 26): 'EIVISSA',
 (7, 32): 'MAHÓN',
 (7, 40): 'PALMA DE MALLORCA',
 (7, 42): 'POLLENÇA',
 (7, 44): 'POBLA (SA)',
 (7, 46): 'SANT ANTONI DE PORTMANY',
 (7, 54): 'SANTA EULAL

In [54]:
# Crear un diccionario vacío para los nombres de las provincias
diccionario_provincias = {}

for index, row in stations_excel.iterrows():
    diccionario_provincias[row['PROVINCIA']] = row['N_PROVINCIA']

diccionario_provincias


{1: 'ÁLAVA',
 2: 'ALBACETE',
 3: 'ALICANTE/ALACANT',
 4: 'ALMERÍA',
 5: 'ÁVILA',
 6: 'BADAJOZ',
 7: 'BALEARS (ILLES)',
 8: 'BARCELONA',
 9: 'BURGOS',
 10: 'CÁCERES',
 11: 'CÁDIZ',
 12: 'CASTELLÓN/CASTELLÓ',
 13: 'CIUDAD REAL',
 14: 'CÓRDOBA',
 15: 'CORUÑA (A)',
 16: 'CUENCA',
 17: 'GIRONA',
 18: 'GRANADA',
 19: 'GUADALAJARA',
 20: 'GUIPÚZCOA',
 21: 'HUELVA',
 22: 'HUESCA',
 23: 'JAÉN',
 24: 'LEÓN',
 25: 'LLEIDA',
 26: 'RIOJA (LA)',
 27: 'LUGO',
 28: 'MADRID',
 29: 'MÁLAGA',
 30: 'MURCIA',
 31: 'NAVARRA',
 32: 'OURENSE',
 33: 'ASTURIAS',
 34: 'PALENCIA',
 35: 'PALMAS (LAS)',
 36: 'PONTEVEDRA',
 37: 'SALAMANCA',
 38: 'SANTA CRUZ DE TENERIFE',
 39: 'CANTABRIA',
 40: 'SEGOVIA',
 41: 'SEVILLA',
 42: 'SORIA',
 43: 'TARRAGONA',
 44: 'TERUEL',
 45: 'TOLEDO',
 46: 'VALENCIA',
 47: 'VALLADOLID',
 48: 'VIZCAYA',
 49: 'ZAMORA',
 50: 'ZARAGOZA',
 51: 'CEUTA'}

In [55]:
dict_ccaa = {(row['PROVINCIA'], row['MUNICIPIO']): row['N_CCAA'] 
                    for index, row in stations_excel.iterrows()}

dict_ccaa

{(1, 22): 'PAÍS VASCO',
 (1, 36): 'PAÍS VASCO',
 (1, 51): 'PAÍS VASCO',
 (1, 55): 'PAÍS VASCO',
 (1, 59): 'PAÍS VASCO',
 (2, 3): 'CASTILLA-LA MANCHA',
 (3, 2): 'COMUNIDAD VALENCIANA',
 (3, 9): 'COMUNIDAD VALENCIANA',
 (3, 14): 'COMUNIDAD VALENCIANA',
 (3, 31): 'COMUNIDAD VALENCIANA',
 (3, 65): 'COMUNIDAD VALENCIANA',
 (3, 66): 'COMUNIDAD VALENCIANA',
 (3, 99): 'COMUNIDAD VALENCIANA',
 (3, 105): 'COMUNIDAD VALENCIANA',
 (3, 133): 'COMUNIDAD VALENCIANA',
 (4, 13): 'ANDALUCÍA',
 (4, 22): 'ANDALUCÍA',
 (4, 24): 'ANDALUCÍA',
 (4, 32): 'ANDALUCÍA',
 (4, 35): 'ANDALUCÍA',
 (4, 66): 'ANDALUCÍA',
 (4, 902): 'ANDALUCÍA',
 (5, 19): 'CASTILLA Y LEÓN',
 (6, 15): 'EXTREMADURA',
 (6, 16): 'EXTREMADURA',
 (6, 22): 'EXTREMADURA',
 (6, 70): 'EXTREMADURA',
 (6, 81): 'EXTREMADURA',
 (6, 83): 'EXTREMADURA',
 (6, 158): 'EXTREMADURA',
 (7, 3): 'BALEARES (ISLAS)',
 (7, 10): 'BALEARES (ISLAS)',
 (7, 15): 'BALEARES (ISLAS)',
 (7, 19): 'BALEARES (ISLAS)',
 (7, 26): 'BALEARES (ISLAS)',
 (7, 32): 'BALEARES (ISLAS)

In [56]:
# Crear clave para agregar la CCAA
pollutants['CLAVE_COMBINADA'] = pollutants.apply(lambda x: (x['PROVINCIA'], x['MUNICIPIO']), axis=1)
pollutants['N_CCAA'] = pollutants['CLAVE_COMBINADA'].map(dict_ccaa)
pollutants.drop('CLAVE_COMBINADA', axis=1, inplace=True)


In [57]:
# Crear el diccionario longitud
dicc_longitude = {(row['PROVINCIA'], row['MUNICIPIO'], row['ESTACION']): row['LONGITUD_G'] for index, row in stations_excel.iterrows()}

# Crear el diccionario latitud
dicc_latitude = {(row['PROVINCIA'], row['MUNICIPIO'], row['ESTACION']): row['LATITUD_G'] for index, row in stations_excel.iterrows()}

In [58]:
# Función para mapear cada fila al valor del diccionario
def mapear_latitud(row):
    # Crea una tupla con los valores de la fila
    clave = (row['PROVINCIA'], row['MUNICIPIO'], row['ESTACION'])
    # Devuelve el valor correspondiente del diccionario
    return dicc_latitude.get(clave, None)  # Retorna None si la clave no existe

# Aplica la función a cada fila y crea una nueva columna
pollutants['LATITUD'] = pollutants.apply(mapear_latitud, axis=1)

In [59]:
# Función para mapear cada fila al valor del diccionario
def mapear_longitud(row):
    # Crea una tupla con los valores de la fila
    clave = (row['PROVINCIA'], row['MUNICIPIO'], row['ESTACION'])
    # Devuelve el valor correspondiente del diccionario
    return dicc_longitude.get(clave, None)  # Retorna None si la clave no existe

# Aplica la función a cada fila y crea una nueva columna
pollutants['LONGITUD'] = pollutants.apply(mapear_longitud, axis=1)

In [60]:
# Crear clave combinada en el DataFrame objetivo para crear la columna con los nombres de los municipios
pollutants['LOC_MUNIC'] = pollutants.apply(lambda x: (x['PROVINCIA'], x['MUNICIPIO']), axis=1)

# Usar el método .map() para mapear los nombres de municipio usando el diccionario.
pollutants['N_MUNICIPIO'] = pollutants['LOC_MUNIC'].map(diccionario_municipios)

pollutants.drop('LOC_MUNIC', axis=1, inplace=True)

In [61]:
# Agregar diccionarios de las magnitudes y provincias
pollutants['MAGNITUD'] = pollutants['MAGNITUD'].map(measured_dict)
pollutants['PROVINCIA'] = pollutants['PROVINCIA'].map(diccionario_provincias)
pollutants

PROVINCIA  MUNICIPIO  ESTACION MAGNITUD   H01   H02   H03   H04   H05  \
0          ÁLAVA         59         8     C6H6  1.58  0.73  0.55  0.68  0.65   
1          ÁLAVA         59         8     C6H6   NaN   NaN   NaN   NaN   NaN   
2          ÁLAVA         59         8     C6H6  0.16  0.16  0.18  0.18  0.14   
3          ÁLAVA         59         8     C6H6  0.23  0.26  0.22  0.20  0.21   
4          ÁLAVA         59         8     C6H6  0.36  0.31  0.32  0.33  0.32   
...          ...        ...       ...      ...   ...   ...   ...   ...   ...   
819609     CEUTA          1         1      SO2  1.64  1.50  1.47  1.44  1.58   
819610     CEUTA          1         1      SO2  2.30  2.15  2.17  2.05  1.82   
819611     CEUTA          1         1      SO2  1.40  1.22  1.22  1.11  1.04   
819612     CEUTA          1         1      SO2  1.68  1.40  1.52  1.66  1.53   
819613     CEUTA          1         1      SO2  3.86  3.54  3.87  4.46  4.78   

         H06  ...   H20    H21   H22   H23   H24      FECHA      N_CCAA  \
0       0.72  ...   NaN    NaN   NaN   NaN   NaN 2022-01-01  PAÍS VASCO   
1        NaN  ...  0.54   0.69  0.32  0.17  0.17 2022-01-03  PAÍS VASCO   
2       0.15  ...  0.28   0.25  0.20  0.20  0.22 2022-01-04  PAÍS VASCO   
3       0.23  ...  0.51   0.51  0.51  0.48  0.41 2022-01-05  PAÍS VASCO   
4       0.33  ...  0.32   0.32  0.30  0.30  0.28 2022-01-06  PAÍS VASCO   
...      ...  ...   ...    ...   ...   ...   ...        ...         ...   
819609  1.38  ...  8.01  11.66  3.28  5.70  5.62 2022-12-16       CEUTA   
819610  1.74  ...  1.65   1.68  1.39  1.30  1.28 2022-12-25       CEUTA   
819611  1.16  ...  2.77   2.66  2.51  2.42  2.52 2022-12-26       CEUTA   
819612  1.57  ...  2.12   1.61  1.57  1.88  1.62 2022-12-28       CEUTA   
819613  4.35  ...  1.97   1.78  1.51   NaN   NaN 2022-12-30       CEUTA   

        LATITUD  LONGITUD      N_MUNICIPIO  
0       42.8548   -2.6807  VITORIA-GASTEIZ  
1       42.8548   -2.6807  VITORIA-GASTEIZ  
2       42.8548   -2.6807  VITORIA-GASTEIZ  
3       42.8548   -2.6807  VITORIA-GASTEIZ  
4       42.8548   -2.6807  VITORIA-GASTEIZ  
...         ...       ...              ...  
819609  35.8930   -5.3147            CEUTA  
819610  35.8930   -5.3147            CEUTA  
819611  35.8930   -5.3147            CEUTA  
819612  35.8930   -5.3147            CEUTA  
819613  35.8930   -5.3147            CEUTA  

[819614 rows x 33 columns]

In [62]:
# Crear el diccionario para el tipo de área
dicc_area = {(row['LATITUD_G'], row['LONGITUD_G']): row['TIPO_AREA'] for index, row in stations_excel.iterrows()}

In [63]:
# Crear el diccionario para el tipo de estación
dicc_tipoestacion = {(row['LATITUD_G'], row['LONGITUD_G']): row['TIPO_ESTACION'] for index, row in stations_excel.iterrows()}

In [64]:
# Función para mapear cada fila al valor del diccionario del tipo de area
def mapear_area(row):
    # Crea una tupla con los valores de la fila
    clave = (row['LATITUD'], row['LONGITUD'])
    # Devuelve el valor correspondiente del diccionario
    return dicc_area.get(clave, None)  # Retorna None si la clave no existe

# Aplica la función a cada fila y crea una nueva columna
pollutants['TIPO_AREA'] = pollutants.apply(mapear_area, axis=1)

In [65]:
# Función para mapear cada fila al valor del diccionario del tipo de estacion
def mapear_estacion(row):
    # Crea una tupla con los valores de la fila
    clave = (row['LATITUD'], row['LONGITUD'])
    # Devuelve el valor correspondiente del diccionario
    return dicc_tipoestacion.get(clave, None)  # Retorna None si la clave no existe

# Aplica la función a cada fila y crea una nueva columna
pollutants['TIPO_ESTACION'] = pollutants.apply(mapear_estacion, axis=1)

In [66]:
# Cambiar el orden de las columnas

# Nombres de las columnas de horas en orden
columnas_horas = ['H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10', 
                  'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20', 
                  'H21', 'H22', 'H23', 'H24']

# Nuevo orden de las columnas
nuevo_orden = ['FECHA', 'N_CCAA', 'PROVINCIA', 'N_MUNICIPIO', 'ESTACION', 'MAGNITUD', 'TIPO_AREA', 'TIPO_ESTACION', 'LATITUD', 'LONGITUD'] + columnas_horas

pollutants = pollutants[nuevo_orden]
pollutants


FECHA      N_CCAA PROVINCIA      N_MUNICIPIO  ESTACION MAGNITUD  \
0      2022-01-01  PAÍS VASCO     ÁLAVA  VITORIA-GASTEIZ         8     C6H6   
1      2022-01-03  PAÍS VASCO     ÁLAVA  VITORIA-GASTEIZ         8     C6H6   
2      2022-01-04  PAÍS VASCO     ÁLAVA  VITORIA-GASTEIZ         8     C6H6   
3      2022-01-05  PAÍS VASCO     ÁLAVA  VITORIA-GASTEIZ         8     C6H6   
4      2022-01-06  PAÍS VASCO     ÁLAVA  VITORIA-GASTEIZ         8     C6H6   
...           ...         ...       ...              ...       ...      ...   
819609 2022-12-16       CEUTA     CEUTA            CEUTA         1      SO2   
819610 2022-12-25       CEUTA     CEUTA            CEUTA         1      SO2   
819611 2022-12-26       CEUTA     CEUTA            CEUTA         1      SO2   
819612 2022-12-28       CEUTA     CEUTA            CEUTA         1      SO2   
819613 2022-12-30       CEUTA     CEUTA            CEUTA         1      SO2   

       TIPO_AREA TIPO_ESTACION  LATITUD  LONGITUD  ...    H15    H16    H17  \
0         URBANA       TRAFICO  42.8548   -2.6807  ...    NaN    NaN    NaN   
1         URBANA       TRAFICO  42.8548   -2.6807  ...   0.23   0.22   0.23   
2         URBANA       TRAFICO  42.8548   -2.6807  ...   0.17   0.18   0.19   
3         URBANA       TRAFICO  42.8548   -2.6807  ...   0.48   0.33   0.45   
4         URBANA       TRAFICO  42.8548   -2.6807  ...   0.30   0.26   0.29   
...          ...           ...      ...       ...  ...    ...    ...    ...   
819609    URBANA         FONDO  35.8930   -5.3147  ...  15.03  25.54  12.58   
819610    URBANA         FONDO  35.8930   -5.3147  ...   2.72   2.27   2.07   
819611    URBANA         FONDO  35.8930   -5.3147  ...   3.31   3.77   2.82   
819612    URBANA         FONDO  35.8930   -5.3147  ...   1.88   1.71   1.66   
819613    URBANA         FONDO  35.8930   -5.3147  ...   2.89    NaN   2.29   

          H18    H19   H20    H21   H22   H23   H24  
0         NaN    NaN   NaN    NaN   NaN   NaN   NaN  
1        0.58   0.24  0.54   0.69  0.32  0.17  0.17  
2        0.25   0.25  0.28   0.25  0.20  0.20  0.22  
3        0.58   0.57  0.51   0.51  0.51  0.48  0.41  
4        0.36   0.43  0.32   0.32  0.30  0.30  0.28  
...       ...    ...   ...    ...   ...   ...   ...  
819609  38.78  17.84  8.01  11.66  3.28  5.70  5.62  
819610   2.00   1.79  1.65   1.68  1.39  1.30  1.28  
819611   2.47   2.79  2.77   2.66  2.51  2.42  2.52  
819612   1.70   1.58  2.12   1.61  1.57  1.88  1.62  
819613   2.10   2.23  1.97   1.78  1.51   NaN   NaN  

[819614 rows x 34 columns]

In [67]:
pollutants = pollutants.dropna()
pollutants

FECHA      N_CCAA PROVINCIA      N_MUNICIPIO  ESTACION MAGNITUD  \
2      2022-01-04  PAÍS VASCO     ÁLAVA  VITORIA-GASTEIZ         8     C6H6   
3      2022-01-05  PAÍS VASCO     ÁLAVA  VITORIA-GASTEIZ         8     C6H6   
4      2022-01-06  PAÍS VASCO     ÁLAVA  VITORIA-GASTEIZ         8     C6H6   
5      2022-01-07  PAÍS VASCO     ÁLAVA  VITORIA-GASTEIZ         8     C6H6   
6      2022-01-08  PAÍS VASCO     ÁLAVA  VITORIA-GASTEIZ         8     C6H6   
...           ...         ...       ...              ...       ...      ...   
819608 2022-12-13       CEUTA     CEUTA            CEUTA         1      SO2   
819609 2022-12-16       CEUTA     CEUTA            CEUTA         1      SO2   
819610 2022-12-25       CEUTA     CEUTA            CEUTA         1      SO2   
819611 2022-12-26       CEUTA     CEUTA            CEUTA         1      SO2   
819612 2022-12-28       CEUTA     CEUTA            CEUTA         1      SO2   

       TIPO_AREA TIPO_ESTACION  LATITUD  LONGITUD  ...    H15    H16    H17  \
2         URBANA       TRAFICO  42.8548   -2.6807  ...   0.17   0.18   0.19   
3         URBANA       TRAFICO  42.8548   -2.6807  ...   0.48   0.33   0.45   
4         URBANA       TRAFICO  42.8548   -2.6807  ...   0.30   0.26   0.29   
5         URBANA       TRAFICO  42.8548   -2.6807  ...   0.28   0.29   0.28   
6         URBANA       TRAFICO  42.8548   -2.6807  ...   0.25   0.20   0.23   
...          ...           ...      ...       ...  ...    ...    ...    ...   
819608    URBANA         FONDO  35.8930   -5.3147  ...   2.22   2.43   3.32   
819609    URBANA         FONDO  35.8930   -5.3147  ...  15.03  25.54  12.58   
819610    URBANA         FONDO  35.8930   -5.3147  ...   2.72   2.27   2.07   
819611    URBANA         FONDO  35.8930   -5.3147  ...   3.31   3.77   2.82   
819612    URBANA         FONDO  35.8930   -5.3147  ...   1.88   1.71   1.66   

          H18    H19    H20    H21    H22    H23    H24  
2        0.25   0.25   0.28   0.25   0.20   0.20   0.22  
3        0.58   0.57   0.51   0.51   0.51   0.48   0.41  
4        0.36   0.43   0.32   0.32   0.30   0.30   0.28  
5        0.32   0.31   0.23   0.25   0.29   0.29   0.28  
6        0.25   0.25   0.23   0.20   0.22   0.22   0.19  
...       ...    ...    ...    ...    ...    ...    ...  
819608   6.62  19.11  17.15   8.29  29.35  24.51  24.16  
819609  38.78  17.84   8.01  11.66   3.28   5.70   5.62  
819610   2.00   1.79   1.65   1.68   1.39   1.30   1.28  
819611   2.47   2.79   2.77   2.66   2.51   2.42   2.52  
819612   1.70   1.58   2.12   1.61   1.57   1.88   1.62  

[715747 rows x 34 columns]

In [68]:
# Guardar dataframe procesado
pollutants_clean = pollutants.to_csv('../data/processed/clean_pollutants.csv', index=False)

In [69]:
pollutants.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715747 entries, 2 to 819612
Data columns (total 34 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   FECHA          715747 non-null  datetime64[ns]
 1   N_CCAA         715747 non-null  object        
 2   PROVINCIA      715747 non-null  object        
 3   N_MUNICIPIO    715747 non-null  object        
 4   ESTACION       715747 non-null  int64         
 5   MAGNITUD       715747 non-null  object        
 6   TIPO_AREA      715747 non-null  object        
 7   TIPO_ESTACION  715747 non-null  object        
 8   LATITUD        715747 non-null  float64       
 9   LONGITUD       715747 non-null  float64       
 10  H01            715747 non-null  float64       
 11  H02            715747 non-null  float64       
 12  H03            715747 non-null  float64       
 13  H04            715747 non-null  float64       
 14  H05            715747 non-null  float64       
 15  H06  

: 